In [1]:
import json
import pandas as pd
from datetime import timedelta
import psycopg2


In [2]:
file_path = r'C:\Users\DY\Documents\GitHub\UFCproject\bronze\ufc-master.csv'

df = pd.read_csv(file_path)

In [3]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date',
       'location', 'country', 'Winner',
       ...
       'finish_details', 'finish_round', 'finish_round_time',
       'total_fight_time_secs', 'r_dec_odds', 'b_dec_odds', 'r_sub_odds',
       'b_sub_odds', 'r_ko_odds', 'b_ko_odds'],
      dtype='object', length=119)

In [4]:
columns_to_drop = ['B_win_by_KO/TKO', 'R_win_by_KO/TKO']

df = df.drop(columns=columns_to_drop)

In [5]:
columns_to_drop = []
for column in df.columns:
    if 'rank' in column:
        columns_to_drop.append(column)

print(columns_to_drop)

['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank', 'better_rank']


In [6]:
df.drop(columns=columns_to_drop, inplace=True)

In [7]:
existing_columns = []
for column in df.columns:
    existing_columns.append(column)


        

print(existing_columns)

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_Submission', 'R_win_by_TKO_Doctor_Stoppage', 'R_wi

In [8]:
with open('connection_string.txt') as file:
    conn_string = file.readline()

In [9]:
conn = psycopg2.connect(conn_string)
if conn.closed == 0:
    print("Connection established")

Connection established


In [11]:
cur = conn.cursor()

for index, row in df.iterrows():
    # Convert row to tuple, handling NaN values appropriately
    record_to_insert = tuple([None if pd.isna(x) else x for x in row])

    try:
        cur.execute(insert_query, record_to_insert)
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error inserting row {index}: {error}")
        break  # Stop the loop to inspect the problematic row

conn.commit()
cur.close()

In [10]:
cur = conn.cursor()

column_count = len(df.columns)
placeholders = ', '.join(['%s'] * column_count)

insert_query = f"""
    INSERT INTO mma_fights (R_fighter, B_fighter, R_odds, B_odds, R_ev, B_ev, date, location, country, Winner, title_bout, weight_class, gender, no_of_rounds, B_current_lose_streak, B_current_win_streak, B_draw, B_avg_SIG_STR_landed, B_avg_SIG_STR_pct, B_avg_SUB_ATT, B_avg_TD_landed, B_avg_TD_pct, B_longest_win_streak, B_losses, B_total_rounds_fought, B_total_title_bouts, B_win_by_Decision_Majority, B_win_by_Decision_Split, B_win_by_Decision_Unanimous, B_win_by_Submission, B_win_by_TKO_Doctor_Stoppage, B_wins, B_Stance, B_Height_cms, B_Reach_cms, B_Weight_lbs, R_current_lose_streak, R_current_win_streak, R_draw, R_avg_SIG_STR_landed, R_avg_SIG_STR_pct, R_avg_SUB_ATT, R_avg_TD_landed, R_avg_TD_pct, R_longest_win_streak, R_losses, R_total_rounds_fought, R_total_title_bouts, R_win_by_Decision_Majority, R_win_by_Decision_Split, R_win_by_Decision_Unanimous, R_win_by_Submission, R_win_by_TKO_Doctor_Stoppage, R_wins, R_Stance, R_Height_cms, R_Reach_cms, R_Weight_lbs, R_age, B_age, lose_streak_dif, win_streak_dif, longest_win_streak_dif, win_dif, loss_dif, total_round_dif, total_title_bout_dif, ko_dif, sub_dif, height_dif, reach_dif, age_dif, sig_str_dif, avg_sub_att_dif, avg_td_dif, empty_arena, constant_1, finish, finish_details, finish_round, finish_round_time, total_fight_time_secs, r_dec_odds, b_dec_odds, r_sub_odds, b_sub_odds, r_ko_odds, b_ko_odds)
    VALUES ({placeholders});
"""
# Prepare a list of tuples from the DataFrame
records_to_insert = [tuple(row) for row in df.to_numpy()]

# Execute the query
try:
    cur.executemany(insert_query, records_to_insert)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()

Error: integer out of range

